# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
#________________starer code_____________________________________#
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=100) #changed to 15 from 1500 for testing
lngs = np.random.uniform(lng_range[0], lng_range[1], size=100) #changed to 15 from 1500 for testing
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
#_________________________end starter code______________________________________#

#cities

82

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
#set up api/ create URL
import json

#target each citi in list "cities" found above
url = "http://api.openweathermap.org/data/2.5/weather?"

#build query url and exception
record_number = 0
count = 1
batch = 1

#define lists to capture from Open Weather Site
city_name = []
city_lat = []
city_long = []
temp = []
humidity = []
country = []
cloudiness = []
windspeed = []
date = []

#loop to create list of found data and grab info from api for dataframe
for i, x  in enumerate(cities):
    #set groups of 50
    if count <= 50:
        count +=1        
    else:
        batch += 1 #set of first 50
        count = 0       
    record_number += 1
    try:
        query_url = url + "appid=" + weather_api_key + "&q=" + x
        print(f'Processing Record {record_number} of set {batch} for City: {x}')
        cities_response = requests.get(query_url).json()
        
        #store data to lists
        city_name.append(cities_response['name'])
        city_lat.append(cities_response['coord']['lat'])
        city_long.append(cities_response['coord']['lon'])
        cloudiness.append(cities_response['clouds']['all'])
        temp.append(cities_response['main']['temp'])
        humidity.append(cities_response['main']['humidity']) 
        country.append(cities_response['sys']['country'])
        windspeed.append(cities_response['wind']['speed'])
        date.append(cities_response['dt'])
    except KeyError:
        print(f'No information found for city: {x}') 

Processing Record 1 of set 1 for City: vaini
Processing Record 2 of set 1 for City: kapaa
Processing Record 3 of set 1 for City: hasaki
Processing Record 4 of set 1 for City: mataura
Processing Record 5 of set 1 for City: pemba
Processing Record 6 of set 1 for City: upernavik
Processing Record 7 of set 1 for City: mnogovershinnyy
Processing Record 8 of set 1 for City: nabire
Processing Record 9 of set 1 for City: saint paul
Processing Record 10 of set 1 for City: itarema
Processing Record 11 of set 1 for City: nanortalik
Processing Record 12 of set 1 for City: baoqing
Processing Record 13 of set 1 for City: puerto ayora
Processing Record 14 of set 1 for City: meulaboh
Processing Record 15 of set 1 for City: taolanaro
No information found for city: taolanaro
Processing Record 16 of set 1 for City: palabuhanratu
No information found for city: palabuhanratu
Processing Record 17 of set 1 for City: souillac
Processing Record 18 of set 1 for City: warrnambool
Processing Record 19 of set 1 fo

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [26]:
cities_dict = {"City Name" : city_name, "Latitude":city_lat, "Longitude":city_long, "Tempature": temp, "Humidity": humidity, "Cloudiness":cloudiness, "Windspeed":windspeed, "Date":date }
cities_df = pd.DataFrame(cities_dict)
cities_df['Date']=pd.to_datetime(cities_df['Date'], unit='s')
cities_df.to_csv("../output_data/Citiesweather.csv", index = False, header = True)

In [27]:
cities_df

,City Name,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Date
0,Vaini,-21.20,-175.20,298.15,78,75,7.70,2020-07-13 01:38:00
1,Kapaa,22.08,-159.32,298.90,83,75,6.70,2020-07-13 01:38:00
2,Hasaki,35.73,140.83,294.05,88,75,2.60,2020-07-13 01:38:00
3,Mataura,-46.19,168.86,280.37,83,83,2.20,2020-07-13 01:38:00
4,Pemba,-12.97,40.52,294.60,87,1,6.55,2020-07-13 01:38:00
...,...,...,...,...,...,...,...,...
71,Sinnamary,5.38,-52.95,298.52,88,66,3.23,2020-07-13 01:38:09
72,Provideniya,64.38,-173.30,282.15,81,40,7.00,2020-07-13 01:38:09
73,Nemuro,43.32,145.57,290.15,77,75,1.50,2020-07-13 01:38:09
74,Airai,-8.93,125.41,293.47,68,100,2.76,2020-07-13 01:38:09


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [29]:
#check for cities with high humidity
len(cities_df.loc[cities_df['Humidity'] >= 100])

#keep those with humidity less than 100
clean_cities_df = cities_df.loc[cities_df['Humidity'] < 100]
clean_cities_df

,City Name,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Date
0,Vaini,-21.20,-175.20,298.15,78,75,7.70,2020-07-13 01:38:00
1,Kapaa,22.08,-159.32,298.90,83,75,6.70,2020-07-13 01:38:00
2,Hasaki,35.73,140.83,294.05,88,75,2.60,2020-07-13 01:38:00
3,Mataura,-46.19,168.86,280.37,83,83,2.20,2020-07-13 01:38:00
4,Pemba,-12.97,40.52,294.60,87,1,6.55,2020-07-13 01:38:00
...,...,...,...,...,...,...,...,...
71,Sinnamary,5.38,-52.95,298.52,88,66,3.23,2020-07-13 01:38:09
72,Provideniya,64.38,-173.30,282.15,81,40,7.00,2020-07-13 01:38:09
73,Nemuro,43.32,145.57,290.15,77,75,1.50,2020-07-13 01:38:09
74,Airai,-8.93,125.41,293.47,68,100,2.76,2020-07-13 01:38:09


In [6]:
#  Get the indices of cities that have humidity over 100%.
#high_humid_df = cities_df.loc[cities_df['Humidity'] >= 100]
#high_humid_index = pd.Index(list(high_humid_df['City Name']))
#high_humid_index.get_loc('City Name')
#slice(1, 3, None)

In [33]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
#condition= cities_df.loc[cities_df['Humidity'] < 100]
clean_cities_df = cities_df.where(cond=cities_df.loc[cities_df['Humidity'] < 100],inplace=False)
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


#print(clean_city_data_df)
print(clean_cities_df)

ValueError: Boolean array expected for the condition, not object

In [8]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv
clean_cities_df.to_csv("../output_data/clean_cities_data.csv", index = False, header = True)

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [9]:
# OPTIONAL: Create a function to create Linear Regression plots

In [10]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression